In [45]:
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

In [52]:
busstop_weight = pd.read_csv("../data/bustop_pop.csv")
all_pop = pd.read_csv("../data/total_data.csv")
all_pop["총인구수"] = all_pop["총인구수"].apply(lambda x: int(x.replace(",", "")))
all_pop["노인(수)"] = all_pop["노인(수)"].apply(lambda x: int(x.replace(",", "")))

city_code = pd.DataFrame([
    {"citycode":38010,"city_name":"창원시"},
    {"citycode":38030,"city_name":"진주시"},
    {"citycode":38050,"city_name":"통영시"},
    {"citycode":38060,"city_name":"사천시"},
    {"citycode":38070,"city_name":"김해시"},
    {"citycode":38080,"city_name":"밀양시"},
    {"citycode":38090,"city_name":"거제시"},
    {"citycode":38100,"city_name":"양산시"},
    {"citycode":38310,"city_name":"의령군"},
    {"citycode":38320,"city_name":"함안군"},
    {"citycode":38330,"city_name":"창녕군"},
    {"citycode":38340,"city_name":"고성군"},
    {"citycode":38350,"city_name":"남해군"},
    {"citycode":38360,"city_name":"하동군"},
    {"citycode":38370,"city_name":"산청군"},
    {"citycode":38380,"city_name":"함양군"},
    {"citycode":38390,"city_name":"거창군"},
    {"citycode":38400,"city_name":"합천군"}
    ])

all_pop = pd.merge(all_pop, city_code, on="city_name", how="left")
all_pop = all_pop[["city_name", "citycode", "총인구수", "노인(수)"]].drop_duplicates()
all_pop = all_pop.rename(columns={"citycode" : "도시코드"})
busstop_weight = pd.merge(busstop_weight, all_pop[[ "도시코드", "총인구수", "노인(수)"]], on="도시코드", how="left")

In [53]:
busstop_weight = busstop_weight.rename(columns={
    "읍면동인구수" : "행정동_노인_인구수", 
    "노인(수)" : "노인_인구수"
    })

In [54]:
busstop_weight.columns

Index(['정류장번호', '정류장명', '위도', '경도', '정보수집일', '모바일단축번호', '도시코드', '도시명', '관리도시명',
       'geometry', 'weight', '행정동_노인_인구수', '총인구수', '노인_인구수'],
      dtype='object')

In [55]:
use_col = ['정류장번호', '정류장명', '위도', '경도', '도시코드', '도시명', 'geometry', 'weight', '행정동_노인_인구수', '총인구수', '노인_인구수']

In [56]:
busstop_weight = busstop_weight[use_col]

In [58]:
busstop_weight

,정류장번호,정류장명,위도,경도,도시코드,도시명,geometry,weight,행정동_노인_인구수,총인구수,노인_인구수
0,CWB379000566,구암고등학교,35.246296,128.593978,38010,경상남도 창원시,POINT (1099529.3099473445 1695090.3215359906),478654.0,410959,1003731,276574
1,CWB379000567,사격장입구,35.244923,128.683156,38010,경상남도 창원시,POINT (1107644.8457543373 1695031.1020157929),464401.0,1934447,1003731,276574
2,CWB379000568,장애인복지회관입구,35.244960,128.673597,38010,경상남도 창원시,POINT (1106775.0680812967 1695024.859684883),732161.0,1934447,1003731,276574
3,CWB379000570,장애인복지회관입구,35.244773,128.673243,38010,경상남도 창원시,POINT (1106743.105090053 1695003.6802969584),266638.0,1934447,1003731,276574
4,CWB379000573,지귀상가,35.244778,128.659437,38010,경상남도 창원시,POINT (1105486.9448440736 1694989.4555102403),1120637.0,1934447,1003731,276574
...,...,...,...,...,...,...,...,...,...,...,...
19023,YSB5166,사과나무집,35.491970,128.989600,38100,경상남도 양산시,POINT (1135114.8887175242 1722807.740568875),3171.0,2291970,356268,90708
19024,YSB5167,이지더원3차,35.322630,129.027700,38100,경상남도 양산시,POINT (1138861.2639738221 1704077.9474447218),147088.0,2291970,356268,90708
19025,YSB5168,반도유보라6차,35.317890,129.027500,38100,경상남도 양산시,POINT (1138851.1887177934 1703551.928831223),147088.0,2291970,356268,90708
19026,YSB5169,양우내안애6차,35.318010,129.023400,38100,경상남도 양산시,POINT (1138478.2608111338 1703559.5004212412),147088.0,2291970,356268,90708
